# Training a neural network on MNIST with Keras



In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from acon import AconC

### Load MNIST


In [ ]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

### Build training pipeline


In [ ]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.experimental.AUTOTUNE)

### Build evaluation pipeline

In [ ]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.experimental.AUTOTUNE)

## Step 2: Create and train the model


In [ ]:
inp = Input(shape=(28, 28, 1), name="input")

x = Conv2D(64, (5, 5), strides=(3, 3), padding="same")(inp)
x = BatchNormalization(fused=True)(x)
x = AconC()(x)

x = Conv2D(64, (5, 5), strides = (3, 3), padding = "same")(x)
x = BatchNormalization(fused = True)(x)
x = AconC()(x)

x = Conv2D(64, (5, 5), strides = (3, 3), padding = "same")(x)
x = BatchNormalization(fused = True)(x)
x = AconC()(x)

x = Flatten()(x)

x = Dropout(0.5)(x)

x = Dense(10)(x)

model = Model(inputs=inp, outputs=x)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=3,
    validation_data=ds_test,
)